In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib

# Generate synthetic data
locations = ["Austin, TX", "San Francisco, CA", "Miami, FL", "Denver, CO", "New York, NY", "Seattle, WA", "Atlanta, GA"]
credit_scores = ["poor", "fair", "good", "very_good", "excellent"]
buy_timelines = ["0–3mo", "3–6mo", "6–12mo", "browsing"]
intents = ["live", "rent", "both"]

def generate_data_point():
    min_price = random.randint(100000, 400000)
    max_price = min_price + random.randint(50000, 300000)
    location = random.choice(locations)
    monthly_income = random.randint(1000, 20000)
    co_buyer = random.choice([True, False])
    monthly_debt = random.randint(0, 15000) if monthly_income > 0 else 0
    outstanding_loans = random.choice([True, False])
    credit_score = random.choices(credit_scores, weights=[0.1, 0.2, 0.3, 0.25, 0.15])[0]
    down_payment = random.randint(0, int(0.3 * max_price))
    include_closing_costs = random.choice([True, False])
    knows_dti = random.choice([True, False])
    buy_timeline = random.choice(buy_timelines)
    home_type = "unspecified"
    intent = random.choice(intents)

    # Calculate DTI
    dti = monthly_debt / monthly_income if monthly_income > 0 else 1.0

    # Heuristic labeling
    if credit_score in ["poor", "fair"] and (dti > 0.5 or down_payment == 0):
        risk = "high"
    elif credit_score in ["very_good", "excellent"] and dti < 0.3 and down_payment >= 0.2 * max_price:
        risk = "low"
    else:
        risk = "medium"

    return {
        "min_price": min_price,
        "max_price": max_price,
        "location": location,
        "monthly_income": monthly_income,
        "co_buyer": co_buyer,
        "monthly_debt": monthly_debt,
        "outstanding_loans": outstanding_loans,
        "credit_score": credit_score,
        "down_payment": down_payment,
        "include_closing_costs": include_closing_costs,
        "knows_dti": knows_dti,
        "buy_timeline": buy_timeline,
        "home_type": home_type,
        "intent": intent,
        "risk": risk
    }

# Generate 500 synthetic samples
data = [generate_data_point() for _ in range(500)]
df = pd.DataFrame(data)

# Define features and target
X = df.drop("risk", axis=1)
y = df["risk"]

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Identify categorical and numerical features
categorical_features = ["location", "credit_score", "buy_timeline", "home_type", "intent"]
numerical_features = [col for col in X.columns if col not in categorical_features]

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features),
    ]
)

# Define the model pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Predict on test set
y_pred = pipeline.predict(X_test)

# Classification report
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

# Save the model and label encoder
model_path = "risk_model.pkl"
label_encoder_path = "label_encoder.pkl"
joblib.dump(pipeline, model_path)
joblib.dump(label_encoder, label_encoder_path)

# Print classification report
print(report)

# Model and label encoder file paths
print(f"Model saved to: {model_path}")
print(f"Label encoder saved to: {label_encoder_path}")


# Load the trained model and label encoder
pipeline = joblib.load("risk_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Sample new data point (in the same format as the synthetic data)
new_data = {
    "min_price": 350000,
    "max_price": 400000,
    "location": "Austin, TX",
    "monthly_income": 10000,
    "co_buyer": True,
    "monthly_debt": 2000,
    "outstanding_loans": True,
    "credit_score": "good",
    "down_payment": 70000,
    "include_closing_costs": False,
    "knows_dti": True,
    "buy_timeline": "3–6mo",
    "home_type": "unspecified",
    "intent": "live"
}

# Convert new data to dataframe
new_df = pd.DataFrame([new_data])

# Predict risk
prediction = pipeline.predict(new_df)
risk_class = label_encoder.inverse_transform(prediction)

print(f"Predicted risk: {risk_class[0]}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

        high       0.85      0.77      0.81        22
         low       0.00      0.00      0.00         5
      medium       0.88      0.96      0.92        73

    accuracy                           0.87       100
   macro avg       0.58      0.58      0.57       100
weighted avg       0.83      0.87      0.85       100

Model saved to: risk_model.pkl
Label encoder saved to: label_encoder.pkl
Predicted risk: medium
